In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# 
import sys
sys.path.append("/home/cat/code/widefieldPredict/") # Adds higher directory to python modules path.

import h5py
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.cross_decomposition import CCA
import torch
from tqdm import tqdm

import os
import time
import warnings
warnings.filterwarnings("ignore")

from locanmf import LocaNMF, postprocess
from locanmf import analysis_fig4 

from scipy import signal
from scipy.fft import fftshift
from scipy.signal import savgol_filter
from numpy.polynomial import polynomial as P
from scipy.signal import argrelmax


Autosaving every 180 seconds


In [9]:

def load_raw_data(spatial_fname, temporal_fname):
    # GRAB AND RECONSTRUCT DATA
    spatial = np.load(spatial_fname) 
    temporal = np.load(temporal_fname)
    temporal = temporal.transpose(0,2,1)

    #
    print (spatial.shape)
    print (temporal.shape)

    #
    print ("reconstructing data: ")
    data = np.matmul( temporal, spatial)
    print (data.shape)

    #
    print ("getting mean of data: ")
    data_mean = data.mean(0)
    print ("data_mean: ", data_mean.shape)
    # compute variance in raw data- not used
    # var = np.var(data2d, axis=0)
    # print ("var: ", var.shape)

    ######################################
    ###### COMPUTE RAW ROI ACTIVITY ######
    ######################################
    data2D = data_mean.reshape(data_mean.shape[0], 128,128)
    print ("Data mean 2D: ", data2D.shape)

    # 
    means = []
    ctr=0
    for id_ in ordered_names:
        area_id = areas[id_]
        idx = np.where(atlas==area_id)
        print (ctr, "areaId: ", area_id, names[id_], idx[0].shape)
        mask = np.zeros((128,128),'float32') + np.nan
        mask[idx] = 1

        temp = data2D*mask
        roi = np.nanmean(temp, axis=1)
        roi = np.nanmean(roi, axis=1)
        means.append(roi)

        ctr+=1   

    #
    raw_means = np.array(means)
    print ("Raw data means: ", raw_means.shape)

    return raw_means



def load_locaNMF_data(fname_locaNMF):
    # order locaNMF components by plot color ORDER in Fig 4A
    ordered_names = np.array([15,0,14,1,   # retrosplenial areas
                              13,2,
                              12,3,
                              11,4,
                              10,5,
                              9,6,
                              8,7])[::-1]


    # load raw data
    try:
        d = np.load(fname_locaNMF)
    except:
        print ("file missing", fname_locaNMF)
        return None, None, None, None, None
        
    locaNMF_temporal = d['temporal_trial']
    locaNMF_temporal_random = d['temporal_random']
    locaNMF_temporal = locaNMF_temporal[:,ordered_names]
    locaNMF_temporal_random = locaNMF_temporal_random[:,ordered_names]
    #print ("locanmf data: ", locaNMF_temporal.shape)

    # 
    areas = d['areas'][ordered_names]
    names = d['names'][ordered_names]
    #print ("original names: ", names.shape)

    #
    atlas = np.load('/media/cat/4TBSSD/yuki/yongxu/atlas_split.npy')
    #print ("atlas: ",atlas.shape)
    # print (areas)
    # print (names)

    print ("  # trials ", locaNMF_temporal.shape)
    #print ("ORDERED NAMES: ", names[ordered_names])

    
    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random


def plot_locanmf_vs_raw(locaNMF_temporal, raw_means):
# FIg 2B locaNMF vs. raw

    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    raw_means_clip = raw_means[:,:900]
    print ('raw temporal_means: ', raw_means.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,raw_means.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(raw_means_clip.shape[1])/30-30
    fig=plt.figure(figsize=(10,10))
    linewidth=3
    for k in range(raw_means.shape[0]):
        ax1=plt.subplot(121)
        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.ylim(-.1,1.25)

        temp1 = raw_means_clip[k]#/np.max(raw_means[k])
        if k==0:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth, 
                     label = 'raw')
        else:
            plt.plot(t,temp1*scale1+k*scale2,c=colors[k], 
                     linewidth= linewidth)

        #plt.plot([-15,0], [scale3+k*scale2, scale3+k*scale2],'--',c='black',alpha=.2)


        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k],
                    label='locaNMF')
        else:
            plt.plot(t,temp2*scale3+k*scale2,'--',
                     linewidth= linewidth, 
                     c=colors[k])

        plt.plot([-15,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    #     # 
    #     ax2=plt.subplot(122)
    #     plt.xlim(-15,0)
    #     temp3 = temp2*scale3-temp1*scale1
    #     plt.plot(t, temp3*scale3+k*scale4,'--', c=colors[k])

    ax1.legend()
    if False:
        plt.savefig('/home/cat/fano.png',dpi=300)
        plt.close()
    else:
        plt.show()       
        

 
        
        
#
def variance_locaNMF(locaNMF_temporal):
    # 
    t = np.arange(locaNMF_temporal.shape[2])/30 - 30
    means = []
    var = []
    #for k in ordered_names:
    for k in range(locaNMF_temporal.shape[1]):
        temp = locaNMF_temporal[:,k].mean(0)
        means.append(temp)

        # 
        temp = np.var(locaNMF_temporal[:,k],axis=0)
        var.append(temp)

    #
    means = np.array(means)#[:,:900]
    var = np.array(var)#[:,:900]
    #print (means.shape, var.shape)

    return means, var



def plot_longitudinal_roi_loca(n_trials, saved_names, all_means):
    print ('n trials: ', n_trials)
    colors = plt.cm.viridis(np.linspace(0,1,len(all_means)))
    area_ids = [0,1,6,7,8,9,12,13]
    time= np.arange(all_means[0].shape[1])/30-30.

    #
    ctr=1
    min_trials = 10
    aucs = []
    saved = []
    fig=plt.figure(figsize=(10,6))
    for ctr, area_id in enumerate(area_ids): 
        ax = plt.subplot(2,4,ctr+1)

        # 
        aucs.append([])
        counter = 0
        for t in range(len(all_means)):
            temp = all_means[t][area_id]
            if n_trials[t]<min_trials:
                continue

            if np.max(np.abs(temp))<0.2:
                plt.plot(time, temp, 
                         color=colors[t],
                        alpha=.8)

                auc = np.nansum(np.abs(temp), axis=0)

                aucs[ctr].append([t,auc])
                counter+=1

        print (ctr, 'area_id: ', area_id, counter)
        # 
        #if ctr==5:

        plt.xticks([])
        plt.yticks([])
        plt.xlim(-15,0)
        plt.title(saved_names[area_id],fontsize=8)

        # cmap = matplotlib.cm.viridis
        #norm = matplotlib.colors.Normalize(vmin=5, vmax=10)

        # cb1 = matplotlib.colorbar.ColorbarBase(ax, cmap=cmap,
         #                               orientation='vertical')

    #
    if False:
        plt.savefig('/home/cat/'+str(animal_id)+'_loca_longitudinal.png',dpi=300)
        plt.close()
    else:
        plt.show()
        
        
def plot_locanmf_temporal_averages(fig, 
                                   locaNMF_temporal,
                                  clr):
    locaNMF_temporal_means_clip = np.mean(locaNMF_temporal,axis=0)[:,:900]
    print ('locaNMF_temporal_means: ', 
            locaNMF_temporal_means_clip.shape)

    # 
    colors = plt.cm.jet(np.linspace(0,1,locaNMF_temporal_means_clip.shape[0]))
    scale1 = 1
    scale2 = scale4 = .075
    scale3 = 1

    #
    t = np.arange(locaNMF_temporal_means_clip.shape[1])/30-30
    linewidth=5
    for k in range(locaNMF_temporal_means_clip.shape[0]):
        #ax1=plt.subplot(121)
        #plt.xticks([])
        #plt.yticks([])
        plt.xlim(-30,0)
        #plt.ylim(-.1,1.25)

        # locanmf traces
        temp2 = locaNMF_temporal_means_clip[k]#/np.max(locaNMF_temporal_means[k])

        if k ==0: 
            plt.plot(t,temp2*scale3+k*scale2,
                     linewidth= linewidth, 
                     c=clr)
        else:
            plt.plot(t,temp2*scale3+k*scale2,
                     linewidth= linewidth, 
                     c=clr)

        plt.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.2)

    
    
##################
def plot_variance_locaNMF(fig, var,
                         clr):

    scale1 = 1
    scale2 = .004
    linewidth=5
    # scale3 = np.nan
    t = np.arange(var.shape[1])/30-30
    
    
    # 
    for k in range(var.shape[0]):
        # 
        plt.xticks([])
        #plt.yticks([])

        # 
        temp = var[k]
        temp = temp*scale1+k*scale2

        # 
        plt.plot(t,temp, c=clr,
                linewidth=linewidth)

        plt.plot([-30,0], [k*scale2,k*scale2],'--',c='black',alpha=.5)

        
        # 
        mean2 = np.mean(temp)
        plt.plot([-30,0], [mean2,mean2],'--',c=clr,alpha=.5)

    # 
    plt.xlim(-30,0)


def load_locaNMF_temporal(animal_id, session_name, root_dir,
                         session_id):

    loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

    #
    loca.get_sessions(session_name)
    print ("sessions: ", loca.sessions.shape)
    print ("selected session: ", loca.sessions[session_id])

    session = loca.sessions[session_id]

    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')


    atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random = load_locaNMF_data(fname_locaNMF)

    return atlas, areas, names, locaNMF_temporal, locaNMF_temporal_random 
    
    

In [40]:
###############################################
########### SHOW PCA OSCILLATIONS #############
###############################################

fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar2_30Hz/IJ1pm_Mar2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
#fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Feb1_30Hz/IJ1pm_Feb1_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
#fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Feb2_30Hz/IJ1pm_Feb2_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Apr6_30Hz/IJ1pm_Apr6_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Apr5_30Hz/IJ1pm_Apr5_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Apr4_30Hz/IJ1pm_Apr4_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Apr1_30Hz/IJ1pm_Apr1_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'

fname = '/media/cat/4TBSSD/yuki/IJ1/tif_files/IJ1pm_Mar31_30Hz/IJ1pm_Mar31_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'



#############
data = np.load(fname)[:,:,:900]
print (data.shape)
random = np.load(fname.replace('trial','random'))[:,:,:900]

#data = data
t = np.arange(data.shape[2])/30-30.

fig=plt.figure()
ax=plt.subplot(121)
plt.plot(t,data[:,0].T,c='black',
        alpha=.1)
mean = np.mean(data[:,0], axis=0)
plt.plot(t,mean)
plt.ylim(-10,10)
plt.xlim(-15,0)
ax=plt.subplot(122)
plt.plot(t,random[:,0].T,c='black',
        alpha=.1)
mean = np.mean(random[:,0], axis=0)
plt.plot(t,mean)
plt.xlim(-15,0)
plt.ylim(-10,10)
plt.show()

(47, 7, 900)


In [3]:
# ###########################################################
# ########### LOAD RAW DATA OPTIONAL - FIG 4A ONLY ##########
# ###########################################################

# spatial_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95_spatial.npy'
# temporal_fname = '/media/cat/4TBSSD/yuki/yongxu/data/IJ1/Mar3/IJ1pm_Mar3_30Hz_code_04_trial_ROItimeCourses_30sec_pca_0.95.npy'
    
# raw_means = load_raw_data(spatial_fname, temporal_fname)

# # Plot locaNMF vs. raw
# plot_locanmf_vs_raw(locaNMF_temporal, raw_means)

In [6]:
######################################################
###### LOAD ANIMAL ALL SESSION MEANS/VARS ############
######################################################

animal_id = 'AQ2'
session_name = 'all'
root_dir = '/media/cat/4TBSSD/yuki/'


# 
loca = analysis_fig4.LocaNMFClass(root_dir, animal_id, session_name)

#
loca.get_sessions(session_name)
print ("# of sessions: ", loca.sessions.shape)

all_means = []
n_trials = []
saved_names = []
all_means_random = []
good_sessions = []
for session in tqdm(loca.sessions):
    
    # load data
    fname_locaNMF = os.path.join(root_dir, animal_id, 'tif_files',session,
                                 session + '_locanmf.npz')
    atlas, areas, names, locaNMF_temporal, random = load_locaNMF_data(fname_locaNMF)

    # 
    if atlas is not None:
        means, var = variance_locaNMF(locaNMF_temporal)
        n_trials.append(locaNMF_temporal.shape[0])
        all_means.append(means)
        
        means, var = variance_locaNMF(random)
        all_means_random.append(means)
                
        saved_names = names
        good_sessions.append(session)
    
print ("# good sessions: ", len(good_sessions))

 14%|█▎        | 15/110 [00:00<00:00, 122.26it/s]

# of sessions:  (110,)
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/AQ2am_Dec9_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec10_30Hz/AQ2am_Dec10_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec10_30Hz/AQ2pm_Dec10_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec11_30Hz/AQ2am_Dec11_30Hz_locanmf.npz
  # trials  (9, 16, 1801)
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec14_30Hz/AQ2am_Dec14_30Hz_locanmf.npz
  # trials  (26, 16, 1801)
  # trials  (21, 16, 1801)
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Dec21_30Hz/AQ2am_Dec21_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki

 25%|██▌       | 28/110 [00:00<00:01, 44.72it/s] 

file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan7_30Hz/AQ2am_Jan7_30Hz_locanmf.npz
  # trials  (27, 16, 1801)
  # trials  (17, 16, 1801)
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan11_30Hz/AQ2am_Jan11_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Jan12_30Hz/AQ2pm_Jan12_30Hz_locanmf.npz
file missing /media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2am_Jan12_30Hz/AQ2am_Jan12_30Hz_locanmf.npz
  # trials  (37, 16, 1801)
  # trials  (54, 16, 1801)


 32%|███▏      | 35/110 [00:00<00:02, 34.81it/s]

...

  # trials  (114, 16, 1801)
  # trials  (112, 16, 1801)


 99%|█████████▉| 109/110 [00:07<00:00,  9.05it/s]

  # trials  (138, 16, 1801)
  # trials  (86, 16, 1801)
  # trials  (82, 16, 1801)


100%|██████████| 110/110 [00:07<00:00, 14.25it/s]

# good sessions:  77


In [7]:
########################################
###### PLOT LONGITUDINAL RESULTS #######
#########################################
# # fig 5F longintudinal power spectra 
# plot_longitudinal_roi_loca(n_trials, saved_names, all_means)   

In [8]:
##############################
####### POWER SPECTRUM #######
##############################


def plot_spectra_longintudinally_two_areas(colors, plotting, fs, selected_areas, 
                                all_areas,
                                all_means,
                                all_means_random):

    #norms = [True, False]
    norms = [False]
    for norm in norms:
        fig = plt.figure(figsize=(12,12))
        #for ctr2, area_id in enumerate(all_areas):
        ctr2=0
        averages = []
        for k in range(len(selected_areas)):
            averages.append([])

        # loop over all sessions
        for k in range(len(all_means)):
            
            # get all traces within a session
            all_traces = all_means[k]
            
            # 
            print (" # areas: ", len(all_traces))
            
            all_traces_random = all_means_random[k]
            # loop over all areas selected
            temp_list = []
            temp_list_random = []
            pxx_list = []
            #pxx_random_list = []
                
            for ctr_area, selected_area in enumerate(selected_areas):
                
                #########################################
                ######### PLOT INDIVIDUAL SPECGRAMS #####
                #########################################
                ax = plt.subplot(1,3,ctr_area+1)
                if norm == False:
                    plt.ylim(1E-8,1E-2)
                else:
                    plt.ylim(1E-3,1E6)

                plt.xlabel('frequency [Hz]')
                plt.xlim(2E-2,6)
                #ax.set_yticks([])
                plt.semilogy()
                plt.semilogx()

                temp = all_traces[selected_area]
                temp_random = all_traces_random[selected_area]
                #temp = savgol_filter(temp, 7, 1)
                temp_list.append(temp)
                temp_list_random.append(temp_random)
                if True: #np.max(np.abs(temp))<0.2:
                    f, Pxx_den = signal.periodogram(temp, fs)
                    f, Pxx_den_random = signal.periodogram(temp_random, fs)

                if norm:
                    Pxx_den = Pxx_den/Pxx_den_random

                # 
                # SHOW ONLY FIRST 451 datapoints
                x = np.arange(0,451,1)/30.
                Pxx_den = Pxx_den[:451]
                #print ('x shape: ', x.shape, Pxx_den.shape)
                
                plt.plot(x, 
                     Pxx_den,
                     linewidth=3,
                     c= colors[ctr_area][ctr2])
                # 
                xx = argrelmax(Pxx_den, order=100)
                #print ("Peak: ", x[xx])

                pxx_list.append(Pxx_den)
                #plt.yticks([])
                averages[ctr_area].append(Pxx_den)
                    
                
            # compute difference
            # Pxx_den = pxx_list[0]-pxx_list[1]

            #########################################
            ######### PLOT RATIO ####################
            #########################################
            if False:
                ax = plt.subplot(1,3,3)

                if False: #np.max(np.abs(temp))<0.2:
                    f, Pxx_den = signal.periodogram(temp_list[0]/temp_list[1], fs)
                    f, Pxx_den_random = signal.periodogram(temp_list_random[0]/temp_list_random[1], fs)

                if True: #np.max(np.abs(temp))<0.2:
                    f, Pxx_den = signal.periodogram(temp_list[0]/temp_list_random[0], fs)
                    #f, Pxx_den_random = signal.periodogram(temp_list_random[0]/temp_list_random[1], fs)


                #if norm:
                #    Pxx_den = Pxx_den/Pxx_den_random

                plt.plot(x, 
                         Pxx_den,
                         linewidth=3,
                         c= colors[2][ctr2])

                if norm == False:
                    plt.ylim(1E-4,1E5)
                else:
                    plt.ylim(1E-3,1E6)

                plt.xlabel('frequency [Hz]')
                plt.xlim(2E-2,6)
                plt.semilogy()
                plt.semilogx()
                #plt.yticks([])

            ctr2+=1
        
        #########################################
        ######### PLOT AVERAGES #################
        #########################################
        for ctr_area in range(2):
            ax = plt.subplot(1,3,ctr_area+1)

            y = np.median(averages[ctr_area],axis=0)
            plt.plot(x, 
                 y,
                 linewidth=3,
                 c= 'black')
                
            #
            plt.plot([0.1, 0.1],[1E-8,1E5], '--', c='black',alpha=.5)
            plt.plot([1, 1],[1E-8,1E5], '--',c='black',alpha=.5)

            for k in range(-8,-2,1):
                plt.plot([1E-4,20],[10**k,10**k], '--',c='black',alpha=.5)

                
            #plt.yticks([])
            plt.xticks([])
            

        #plt.ylabel('PSD [V**2/Hz]')
        #plt.title(saved_names[area_id], fontsize=8)
        #plt.xlim(0,2)
        #plt.yticks([])

        if False:
            plt.savefig('/home/cat/spectra_'+str(animal_id)+'norm_'+str(norm)+'.png',dpi=300)
            plt.close()
        else:
            plt.show()
            
            
#
all_areas = np.arange(all_means[0].shape[0])
#area_ids = [0,1,6,7,8,9,12,13]
#area_ids = [7]


#colors = plt.cm.viridis(np.linspace(0,1,len(all_areas)))
colors = []
colors.append(plt.cm.Oranges(np.linspace(0,1,len(all_means))))
colors.append(plt.cm.Greens(np.linspace(0,1,len(all_means))))
colors.append(plt.cm.Greys(np.linspace(0,1,len(all_means))))

fs = 30

plotting=False

selected_areas = [9,11]

plot_spectra_longintudinally_two_areas(colors, plotting, fs, selected_areas, 
                                        all_areas,
                                        all_means,
                                        all_means_random)



 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # areas:  16
 # are